In [1]:
import pandas as pd
import numpy as np

In [46]:
# === 读取原始数据 ===
players = pd.read_csv("data/players_preprocessed_cleaned.csv")
teams = pd.read_csv("data/teams_preprocessed.csv")
# players

In [47]:
# 读取 play-team.csv 文件
df = pd.read_csv('data/play-team.csv')
df = df.rename(columns={'Year': 'Season_End_Year'})
# 1️⃣ 保留英超数据
df_epl = df[df['Comp'] == 'Premier League']

# 2️⃣ 合并 player 数据
players = players.merge(
    df_epl[['Season_End_Year', 'Player', 'Squad']],  # 只保留关键字段
    on=['Season_End_Year', 'Player'],              # 按这两列合并
    how='left'                                    # 左连接：保留 player 所有数据
)

players

,Season_End_Year,Player,Nation,Pos,Age,Born,MP_Playing,Starts_Playing,Min_Playing,Mins_Per_90_Playing,...,xAG_Expected,npxG_plus_xAG_Expected,xG_Per_90,xAG_Per_90,xG_plus_xAG_Per_90,npxG_Per_90,npxG_plus_xAG_Per_90,Year,market_value_euro,Squad
0,2020,Pierre-Emerick Aubameyang,GAB,FW,30,1989,36,35,3136,34.8,...,3.3,16.5,0.42,0.09,0.52,0.38,0.47,2020,25000000.0,Arsenal
1,2020,Héctor Bellerín,ESP,"DF,MF",24,1995,15,13,1158,12.9,...,0.4,0.8,0.03,0.03,0.06,0.03,0.06,2020,25000000.0,Arsenal
2,2020,Dani Ceballos,ESP,MF,22,1996,24,18,1692,18.8,...,1.8,2.3,0.03,0.09,0.12,0.03,0.12,2020,27000000.0,Arsenal
3,2020,Calum Chambers,ENG,DF,24,1995,14,13,1102,12.2,...,1.6,2.4,0.06,0.13,0.20,0.06,0.20,2020,12000000.0,Arsenal
4,2020,Rob Holding,ENG,DF,23,1995,8,6,581,6.5,...,0.0,0.2,0.03,0.00,0.03,0.03,0.03,2020,18000000.0,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,2024,Matheus Nunes,POR,MF,25-277,1998,2,2,179,2.0,...,0.1,0.2,0.04,0.05,0.09,0.04,0.09,2024,35000000.0,Wolves
1984,2024,José Sá,POR,GK,31-134,1993,35,35,3037,33.7,...,0.0,0.0,0.00,0.00,0.00,0.00,0.00,2024,7000000.0,Wolves
1985,2024,Pablo Sarabia,ESP,"MF,FW",32-019,1992,30,20,1745,19.4,...,5.9,9.8,0.28,0.30,0.59,0.20,0.50,2024,5000000.0,Wolves
1986,2024,Nélson Semedo,POR,"DF,MF",30-196,1993,36,36,3084,34.3,...,1.7,3.0,0.04,0.05,0.09,0.04,0.09,2024,9000000.0,Wolves


In [50]:
# players['Age'] = players['Age'][:2]
players['Age'] = players['Age'].astype(str).str.extract(r'(\d{1,2})').astype('Int64')
# players.Age.value_counts()

In [51]:
# ----------------------------------------------------------
# 🧩 1. 球员层特征衍生
# ----------------------------------------------------------

# --- 基础标志类 ---
# 是否为英格兰球员
players["is_eng"] = (players["Nation"] == "ENG").astype(int)

# Big6 球队名单（可根据赛季灵活修改）
big6 = ["Arsenal", "Chelsea", "Liverpool", "Manchester United", "Manchester City", "Tottenham Hotspur"]
players["is_big6_team"] = players["Squad"].isin(big6).astype(int)

# --- 年龄分层 ---
def age_group(age):
    if age <= 22:
        return "18-22"
    elif age <= 26:
        return "23-26"
    elif age <= 30:
        return "27-30"
    else:
        return "31+"

players = players.apply(pd.to_numeric, errors='ignore')
teams = teams.apply(pd.to_numeric, errors='ignore')
print(type(players["Age"].iloc[0]))

players["age_group"] = players["Age"].apply(age_group)

# --- 球队进攻核心标志 ---
# 定义一个简单的判定规则：进球占比或xG占比前 20% 视为核心球员
# 注意：如果不同年份数据混在一起，需要 groupby Year
players["goal_contrib"] = players["Gls"] / players.groupby(["Year", "Squad"])["Gls"].transform("sum")
players["xG_contrib"] = players["xG_Expected"] / players.groupby(["Year", "Squad"])["xG_Expected"].transform("sum")

# 如果一个球员在进球或xG贡献中任一项占比>0.2（20%），定义为核心
players["is_attack_core"] = ((players["goal_contrib"] > 0.2) | (players["xG_contrib"] > 0.2)).astype(int)

# --- 年龄 × 表现（潜力类特征） ---
players["age_x_market_value"] = players["Age"] * players["market_value_euro"]
players["log_market_value"] = np.log1p(players["market_value_euro"])

# --- 每90分钟产出特征（效率类）---
# 注意：部分球员可能无数据或为0分钟，因此加1防止除零
players["goal_per90"] = players["Gls"] / (players["Min_Playing"] / 90 + 1e-6)
players["assist_per90"] = players["Ast"] / (players["Min_Playing"] / 90 + 1e-6)
players["xG_per90"] = players["xG_Expected"] / (players["Min_Playing"] / 90 + 1e-6)
players["xAG_per90"] = players["xAG_Expected"] / (players["Min_Playing"] / 90 + 1e-6)

# --- 表现相对球队平均（归一化）---
# 例如：球员进球与球队平均的比值（衡量角色强度）
for col in ["Gls", "Ast", "xG_Expected", "xAG_Expected"]:
    team_mean = players.groupby(["Year", "Squad"])[col].transform("mean")
    players[f"{col}_to_team_mean"] = players[col] / (team_mean + 1e-6)

players

<class 'numpy.int64'>


/var/folders/2b/6616n7c16y76cwp79zt5nxvc0000gn/T/ipykernel_62855/513144001.py:24: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  players = players.apply(pd.to_numeric, errors='ignore')
/var/folders/2b/6616n7c16y76cwp79zt5nxvc0000gn/T/ipykernel_62855/513144001.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  teams = teams.apply(pd.to_numeric, errors='ignore')


,Season_End_Year,Player,Nation,Pos,Age,Born,MP_Playing,Starts_Playing,Min_Playing,Mins_Per_90_Playing,...,age_x_market_value,log_market_value,goal_per90,assist_per90,xG_per90,xAG_per90,Gls_to_team_mean,Ast_to_team_mean,xG_Expected_to_team_mean,xAG_Expected_to_team_mean
0,2020,Pierre-Emerick Aubameyang,GAB,FW,30,1989,36,35,3136,34.8,...,750000000.0,17.034386,0.631378,0.086097,0.421875,0.094707,10.999995,2.272726,8.647054,2.915192
1,2020,Héctor Bellerín,ESP,"DF,MF",24,1995,15,13,1158,12.9,...,600000000.0,17.034386,0.077720,0.000000,0.031088,0.031088,0.500000,0.000000,0.235294,0.353357
2,2020,Dani Ceballos,ESP,MF,22,1996,24,18,1692,18.8,...,594000000.0,17.111347,0.000000,0.106383,0.031915,0.095745,0.000000,1.515150,0.352941,1.590105
3,2020,Calum Chambers,ENG,DF,24,1995,14,13,1102,12.2,...,288000000.0,16.300417,0.081670,0.081670,0.065336,0.130671,0.500000,0.757575,0.470588,1.413426
4,2020,Rob Holding,ENG,DF,23,1995,8,6,581,6.5,...,414000000.0,16.705882,0.000000,0.000000,0.030981,0.000000,0.000000,0.000000,0.117647,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983,2024,Matheus Nunes,POR,MF,25,1998,2,2,179,2.0,...,875000000.0,17.370859,0.000000,0.000000,0.050279,0.050279,0.000000,0.000000,0.055138,0.074576
1984,2024,José Sá,POR,GK,31,1993,35,35,3037,33.7,...,217000000.0,15.761421,0.000000,0.029635,0.000000,0.000000,0.000000,0.666666,0.000000,0.000000
1985,2024,Pablo Sarabia,ESP,"MF,FW",32,1992,30,20,1745,19.4,...,160000000.0,15.424949,0.206304,0.361032,0.283668,0.304298,2.588234,4.666664,3.032580,4.399997
1986,2024,Nélson Semedo,POR,"DF,MF",30,1993,36,36,3084,34.3,...,270000000.0,16.012735,0.000000,0.029183,0.037938,0.049611,0.000000,0.666666,0.716792,1.267796


In [52]:
# ----------------------------------------------------------
# 🏟️ 2. 球队层特征衍生（聚合得到）
# ----------------------------------------------------------

# 数值特征聚合（均值、方差、极值）
numeric_cols = ["Gls", "Ast", "xG_Expected", "xAG_Expected", "market_value_euro"]
agg_funcs = ['mean', 'std', 'max', 'sum']

team_features = players.groupby(["Year", "Squad"])[numeric_cols].agg(agg_funcs)
team_features.columns = [f"{col}_{stat}" for col, stat in team_features.columns]
team_features.reset_index(inplace=True)

# 球员层比例聚合
core_ratio = players.groupby(["Year", "Squad"])["is_attack_core"].mean().reset_index(name="attack_core_ratio")
eng_ratio = players.groupby(["Year", "Squad"])["is_eng"].mean().reset_index(name="eng_player_ratio")

# 合并至球队特征表
teams = teams.merge(team_features, on=["Year", "Squad"], how="left")
teams = teams.merge(core_ratio, on=["Year", "Squad"], how="left")
teams = teams.merge(eng_ratio, on=["Year", "Squad"], how="left")
teams

,Year,Squad,Age,Poss,MP_Playing,Starts_Playing,Min_Playing,Mins_Per_90_Playing,Gls,Ast,...,xAG_Expected_mean,xAG_Expected_std,xAG_Expected_max,xAG_Expected_sum,market_value_euro_mean,market_value_euro_std,market_value_euro_max,market_value_euro_sum,attack_core_ratio,eng_player_ratio
0,2019,Arsenal,26.7,58.1,38,418,3420,38,69,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Bournemouth,26.6,46.9,38,418,3420,38,55,43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,Brighton,27.4,42.4,38,418,3420,38,35,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Burnley,28.0,41.2,38,418,3420,38,43,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Cardiff City,27.7,35.4,38,418,3420,38,33,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,2024,Nott'ham Forest,25.7,40.6,38,418,3420,38,49,43,...,1.845455,2.527699,8.4,40.6,2.179545e+07,1.606064e+07,55000000.0,479500000.0,0.045455,0.181818
116,2024,Sheffield Utd,25.7,35.8,38,418,3420,38,31,18,...,0.500000,1.023067,2.8,3.5,6.514286e+06,4.248697e+06,12000000.0,45600000.0,0.142857,0.857143
117,2024,Tottenham,25.2,61.6,38,418,3420,38,69,58,...,2.050000,2.811349,8.1,41.0,2.835000e+07,1.563490e+07,50000000.0,567000000.0,0.050000,0.200000
118,2024,West Ham,28.3,41.1,38,418,3420,38,58,40,...,1.823810,2.682146,7.8,38.3,1.329524e+07,1.209692e+07,45000000.0,279200000.0,0.047619,0.571429


In [53]:
# ----------------------------------------------------------
# 💾 3. 保存结果
# ----------------------------------------------------------
players.to_csv("data/players_features_sy.csv", index=False)
teams.to_csv("data/teams_features_sy.csv", index=False)

print("✅ 特征衍生完成：")
print(f"球员特征数：{players.shape[1]}  球队特征数：{teams.shape[1]}")

✅ 特征衍生完成：
球员特征数：54  球队特征数：54
